In [2]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [4]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [5]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [6]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file

columns = df_market_data[["price_change_percentage_24h","price_change_percentage_7d","price_change_percentage_14d","price_change_percentage_30d","price_change_percentage_60d"]]

market_data_scaled = StandardScaler().fit_transform(columns)


In [12]:
# Create a DataFrame with the scaled data

df_market_scaled = pd.DataFrame(market_data_scaled, columns=["price_change_percentage_24h","price_change_percentage_7d","price_change_percentage_14d","price_change_percentage_30d","price_change_percentage_60d"])


# Copy the crypto names from the original data

df_market_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index

df_market_scaled = df_market_scaled.set_index("coin_id")

# Display sample data

df_market_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d
coin_id,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203


---

### Find the Best Value for k Using the Original Data.

In [13]:
# Create a list with the number of k-values from 1 to 11

k = list(range(1,11))

In [14]:
# Create an empty list to store the inertia values

inertia = []


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_scaled)
    inertia.append(model.inertia_)
    

In [15]:
# Create a dictionary with the data to plot the Elbow curve

elbow_data = {
    "k": k,
    "inertia": inertia
}


# Create a DataFrame with the data to plot the Elbow curve

df_elbow_data = pd.DataFrame(elbow_data)

In [16]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

elbow_plot = df_elbow_data.hvplot.line(
    x= "k",
    y="inertia",
    title="Elbow curve for Market Data",
    xticks=k
)

elbow_plot


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

Best value for k = 3

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [26]:
# Initialize the K-Means model using the best value for k

model = KMeans(n_clusters=3)


In [27]:
# Fit the K-Means model using the scaled data

model.fit(df_market_scaled)

KMeans(n_clusters=3)

In [28]:
# Predict the clusters to group the cryptocurrencies using the scaled data

market_clusters = model.predict(df_market_scaled)


# Print the resulting array of cluster values.

print(market_clusters)


[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 2
 0 0 0 0]


In [29]:
# Create a copy of the DataFrame

market_data_predictions = df_market_scaled.copy()


In [30]:
# Add a new column to the DataFrame with the predicted clusters

market_data_predictions["predictions"] = market_clusters


# Display sample data

market_data_predictions.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,predictions
coin_id,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,1
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,1
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,1


In [57]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

market_data_plot = market_data_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="predictions", 
    hover_cols=["coin_id"]
)

market_data_plot

:NdOverlay   [predictions]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [32]:
# Create a PCA model instance and set `n_components=3`.

pca = PCA(n_components=3)

In [36]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.

market_pca_data = pca.fit_transform(df_market_scaled)

# View the first five rows of the DataFrame. 

market_pca_df = pd.DataFrame(market_pca_data, columns=["PC1", "PC2", "PC3"], index=df_market_data.index)

market_pca_df[:5]


,PC1,PC2,PC3
coin_id,,,
bitcoin,0.795104,-0.662906,-0.153885
ethereum,0.413769,-1.047773,0.157623
tether,-0.195508,0.518261,-0.201535
ripple,-0.260748,0.340601,-0.145741
bitcoin-cash,1.961019,-2.239724,-0.182981


In [37]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.

pca.explained_variance_ratio_


array([0.47862164, 0.26608254, 0.1684978 ])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

The total explained variance is: 0.9132019800000001

In [44]:
pca_sum = sum([0.47862164, 0.26608254, 0.1684978])

pca_sum

0.9132019800000001

In [ ]:
# Create a new DataFrame with the PCA data.



# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data


# Set the coinid column as index


# Display sample data


---

### Find the Best Value for k Using the PCA Data

In [45]:
# Create a list with the number of k-values from 1 to 11

k = list(range(1,11))

In [46]:
# Create an empy list to store the inertia values

inertia = []


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(market_pca_df)
    inertia.append(model.inertia_)


In [48]:
# Create a dictionary with the data to plot the Elbow curve

elbow_market_pca = {
    "k": k,
    "inertia": inertia
} 

# Create a DataFrame with the data to plot the Elbow curve

df_elbow_pca = pd.DataFrame(elbow_market_pca)


In [60]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

elbow_plot_pca = df_elbow_pca.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow curve for PCA Data",
    xticks=k
)

elbow_plot_pca


:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  Best value is k = 3


* **Question:** Does it differ from the best k value found using the original data?

  No, it does not differ from best k value using the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [50]:
# Initialize the K-Means model using the best value for k

model = KMeans(n_clusters=3)

In [51]:
# Fit the K-Means model using the PCA data

model.fit(market_pca_df)

KMeans(n_clusters=3)

In [52]:
# Predict the clusters to group the cryptocurrencies using the PCA data

# Print the resulting array of cluster values.

pca_clusters = model.predict(market_pca_df)

print(pca_clusters)


[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 0 2 0 0 1
 0 0 0 0]


In [53]:
# Create a copy of the DataFrame with the PCA data

df_pca_predictions = market_pca_df.copy()


# Add a new column to the DataFrame with the predicted clusters

df_pca_predictions["Predictions"] = pca_clusters

# Display sample data

df_pca_predictions.head()

,PC1,PC2,PC3,Predictions
coin_id,,,,
bitcoin,0.795104,-0.662906,-0.153885,2
ethereum,0.413769,-1.047773,0.157623,2
tether,-0.195508,0.518261,-0.201535,0
ripple,-0.260748,0.340601,-0.145741,0
bitcoin-cash,1.961019,-2.239724,-0.182981,2


In [58]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

pca_data_plot = df_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Predictions",
    hover_cols=["coin_id"]
)

pca_data_plot

:NdOverlay   [Predictions]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [61]:
# Composite plot to contrast the Elbow curves

(elbow_plot + elbow_plot_pca ).cols(1)

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [59]:
# Composite plot to contrast the clusters

(market_data_plot + pca_data_plot ).cols(1)

:Layout
   .NdOverlay.I  :NdOverlay   [predictions]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Predictions]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 